<a href="https://colab.research.google.com/github/aruntakhur/SitareUniversity/blob/main/MinMax_Search_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab Assignment: Tic Tac Toe with Minimax Search

## Objective
In this lab, you will:
- Implement Tic Tac Toe using Python.
- Write state functions (`S0`, `Player`, `Actions`, `Result`, `Terminal`, `Utility`).
- Implement the **Minimax algorithm**.
- Count the number of states evaluated.
- Compare theoretical vs actual states explored.

In [1]:
# -----------------------------
# Part A: State Representation
# -----------------------------

# Initial state (S0): empty 3x3 board
S0 = [" " for _ in range(9)]


def print_board(state):
    """Print the board in a 3x3 format"""
    print(state[0]+"|"+state[1]+"|"+state[2])
    print("-+-+-")
    print(state[3]+"|"+state[4]+"|"+state[5])
    print("-+-+-")
    print(state[6]+"|"+state[7]+"|"+state[8])



In [2]:
# -----------------------------
# TODO Part A: Implement State Functions
# -----------------------------

def Player(state):
    """Return which player has the next turn"""
    # TODO: Implement using counts of X and O
    x_count = state.count("X")
    o_count = state.count("O")
    return "X" if x_count == o_count else "O"


def Actions(state):
    """Return list of legal moves"""
    # TODO: Return all indexes that are still empty
    return [i for i, cell in enumerate(state) if cell == " "]
    


def Result(state, action):
    """Return new state after action is taken"""
    # TODO: Copy the state and apply the move
    new_state = state.copy()
    new_state[action] = Player(state)
    return new_state


def Terminal(state):
    """Return True if the state is terminal (win or draw)"""
    # TODO: Check for win or if no moves left
    wins = [
        [0,1,2], [3,4,5], [6,7,8],  # rows
        [0,3,6], [1,4,7], [2,5,8],  # columns
        [0,4,8], [2,4,6]            # diagonals
    ]
    for combo in wins:
        if state[combo[0]] != " " and state[combo[0]] == state[combo[1]] == state[combo[2]]:
            return True  # win
    if " " not in state:
        return True  # draw
    return False
    


def Utility(state):
    """Return +1 if X wins, -1 if O wins, else 0"""
    # TODO: Implement win/draw logic
    wins = [
        [0,1,2], [3,4,5], [6,7,8],  # rows
        [0,3,6], [1,4,7], [2,5,8],  # columns
        [0,4,8], [2,4,6]            # diagonals
    ]
    for combo in wins:
        if state[combo[0]] != " " and state[combo[0]] == state[combo[1]] == state[combo[2]]:
            return 1 if state[combo[0]] == "X" else -1
    return 0


In [4]:
# -----------------------------
# Part B: Minimax Implementation
# -----------------------------


def MAX_VALUE(state):

    if Terminal(state):
        return Utility(state)

    v = float("-inf")
    for action in Actions(state):
        v = max(v, MIN_VALUE(Result(state, action)))
    return v


def MIN_VALUE(state):

    if Terminal(state):
        return Utility(state)

    v = float("inf")
    for action in Actions(state):
        v = min(v, MAX_VALUE(Result(state, action)))
    return v


def minimax_decision(state):
    """Return best move for current player"""
    global call_count
    call_count = 0

    player = Player(state)
    best_move = None

    if player == "X":
        best_val = float("-inf")
        for action in Actions(state):
            val = MIN_VALUE(Result(state, action))
            if val > best_val:
                best_val = val
                best_move = action
    else:
        best_val = float("inf")
        for action in Actions(state):
            val = MAX_VALUE(Result(state, action))
            if val < best_val:
                best_val = val
                best_move = action

    return best_move


In [5]:
# -----------------------------
# Part C: Human vs AI Game
# -----------------------------

# Add Code to detect invalid move also

def play():
    state = S0.copy()

    while not Terminal(state):
        print_board(state)
        if Player(state) == "X":  # Human move
            move = int(input("Enter your move (1-9): ")) - 1

        else:  # AI move
            print("AI is thinking...")
            move = minimax_decision(state)

        state = Result(state, move)

    print_board(state)
    if Utility(state) == 1:
        print("X wins!")
    elif Utility(state) == -1:
        print("O wins!")
    else:
        print("It's a draw!")


In [10]:
# -----------------------------
# Part D: Minimax with Alpha-Beta Pruning
# -----------------------------

def MAX_VALUE_AB(state, alpha, beta):
    if Terminal(state):
        return Utility(state)
    
    v = float("-inf")
    for action in Actions(state):
        v = max(v, MIN_VALUE_AB(Result(state, action), alpha, beta))
        if v >= beta:
            return v
        alpha = max(alpha, v)
    return v


def MIN_VALUE_AB(state, alpha, beta):
    if Terminal(state):
        return Utility(state)
    
    v = float("inf")
    for action in Actions(state):
        v = min(v, MAX_VALUE_AB(Result(state, action), alpha, beta))
        if v <= alpha:
            return v
        beta = min(beta, v)
    return v


def minimax_ab_decision(state):
    """Return best move using minimax with alpha-beta pruning."""
    player = Player(state)
    best_move = None
    
    if player == "X":
        best_val = float("-inf")
        alpha, beta = float("-inf"), float("inf")
        for action in Actions(state):
            val = MIN_VALUE_AB(Result(state, action), alpha, beta)
            if val > best_val:
                best_val = val
                best_move = action
            alpha = max(alpha, best_val)
    else:
        best_val = float("inf")
        alpha, beta = float("-inf"), float("inf")
        for action in Actions(state):
            val = MAX_VALUE_AB(Result(state, action), alpha, beta)
            if val < best_val:
                best_val = val
                best_move = action
            beta = min(beta, best_val)
    
    return best_move


# -----------------------------
# Part D: Experiment and Analysis
# -----------------------------

1. Run the game where X plays first in the center (position 5).
2. Note down the number of states AI explored.
3. Compare with:
   - 8! = 40,320 (leaf-only sequences)
   - 8+8⋅7+8⋅7⋅6+⋯+8! = 109,600 (all partial states)
4. Explain why actual count (~55,504) is smaller than 109,600.


In [11]:
# -----------------------------
# Part E: Human vs AI Game (with Alpha-Beta)
# -----------------------------

def play_ab():
    state = S0.copy()
    
    while not Terminal(state):
        print_board(state)
        
        if Player(state) == "X":  # Human move
            try:
                move = int(input("Enter your move (1-9): ")) - 1
                if move not in Actions(state):
                    print("❌ Invalid move, try again.")
                    continue
            except ValueError:
                print("❌ Please enter a number between 1 and 9.")
                continue
        else:  # AI move with alpha-beta
            print("🤖 AI (Alpha-Beta) is thinking...")
            move = minimax_ab_decision(state)
            print("AI chooses position:", move+1)

        state = Result(state, move)

    # Final board and result
    print_board(state)
    if Utility(state) == 1:
        print("🎉 X wins!")
    elif Utility(state) == -1:
        print("🤖 O wins!")
    else:
        print("🤝 It's a draw!")


In [12]:
play()

 | | 
-+-+-
 | | 
-+-+-
 | | 
 | | 
-+-+-
 |X| 
-+-+-
 | | 
🤖 AI is thinking...
AI chooses position: 1
O| | 
-+-+-
 |X| 
-+-+-
 | | 
O| | 
-+-+-
 |X| 
-+-+-
 |X| 
🤖 AI is thinking...
AI chooses position: 2
O|O| 
-+-+-
 |X| 
-+-+-
 |X| 
O|O|X
-+-+-
 |X| 
-+-+-
 |X| 
🤖 AI is thinking...
AI chooses position: 7
O|O|X
-+-+-
 |X| 
-+-+-
O|X| 
O|O|X
-+-+-
X|X| 
-+-+-
O|X| 
🤖 AI is thinking...
AI chooses position: 6
O|O|X
-+-+-
X|X|O
-+-+-
O|X| 
O|O|X
-+-+-
X|X|O
-+-+-
O|X|X
🤝 It's a draw!


# -----------------------------
# Part E: Written Questions
# -----------------------------

Answer briefly:

1. Why does Minimax need to evaluate so many states in Tic Tac Toe?
2. Difference between full tree size and actual explored states?
3. How do early wins reduce the number of states?
4. What challenges would arise if applying Minimax to Chess?
